In [1]:
!pip install spotipy
import spotipy
import json
import requests
import re
import html
import os
import time

from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
client_id = '3ce1f5cf8ebf4d33aef320f9c4834037'
client_secret = '94476eff9d0a410a85b7793d24a11c83'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

#[R&B -> Usher, Rap -> Eminem, Rock -> Pink Floyd, Country -> Morgan Wallen, Jazz -> Frank Sinatra, K-Pop -> BTS, Reggae -> Bob Marley, 
# Gospel -> Kirk Franklin, Funk -> James Brown] 
initial_genre_artist = {'R&B': ('Usher','23zg3TcAtWQy7J6upgbUnj'), 'Hip-Hop': ('Eminem','7dGJo4pcD2V6oG8kP0tJRR'), 'Rock': ('Pink Floyd','36QJpDe2go2KgaRleHCDTp'), 'Country': ('Morgan Wallen','4oUHIQIBe0LHzYfvXNW4QM'), 'Jazz': ('Frank Sinatra','1Mxqyy3pSjf8kZZL4QVxS0'), 'K-Pop': ('BTS','3Nrfpe0tUJi4K4DXYWgMUX'),'Reggae': ('Bob Marley and The Wailers','2QsynagSdAqZj3U9HgDzjD'),'Gospel': ('Kirk Franklin','4akybxRTGHJZ1DXjLhJ1qu'),'Funk': ('James Brown','7GaxyUddsPok8BuhxN6OUW')}

#genre -> artist IDs -> songs
song_info = {}

for genre, artist in initial_genre_artist.items():
    
    related_artist_data = sp.artist_related_artists(artist[1])['artists']
    song_info[genre] = {}
    song_info[genre][artist] = set()
    
    for rel in related_artist_data:
        song_info[genre][(rel['name'],rel['id'])] = set()
    
count = 0
for genre, artists in song_info.items():
    for (name, a_id), songs in artists.items():
        albums = sp.artist_albums(a_id, limit=5)['items']

        for ind_album in albums:
            album_id = ind_album['id']
            tracks = sp.album_tracks(album_id, limit=10)['items']
            for track in tracks:
                songs.add(track['name'])
                count += 1
                
print('Number of songs: {}'.format(count))

Number of songs: 8595


In [17]:
#songs_to_genre = []
#'http://41.65.242.43:1976'
proxy_url = "https://proxy-orbit1.p.rapidapi.com/v1/"
headers = {
    'x-rapidapi-host': "proxy-orbit1.p.rapidapi.com",
    'x-rapidapi-key': "723c47a9f5msh1ddd8d271a9f246p1c967fjsnf248ea726d5e"
}
proxy_response = requests.get(proxy_url, headers=headers)
proxy_json = proxy_response.json()
print(proxy_json)
proxy = proxy_json['curl']


{'anonymous': True, 'cookies': True, 'curl': 'http://85.195.95.222:1080', 'get': True, 'ip': '85.195.95.222', 'isp': 'Host Europe GmbH', 'lastChecked': 1638916085.7099514, 'location': None, 'port': 1080, 'post': True, 'protocol': 'http', 'rtt': 2.1126058101654053, 'ssl': True, 'websites': {'amazon': True, 'facebook': True, 'google': True, 'instagram': True, 'netflix': True, 'nike': True, 'shopify': True, 'stubhub': True, 'supreme': True, 'ticketmaster': True, 'twitch': True, 'twitter': True, 'youtube': True}}


In [46]:
songs_per_genre = 100
failed_song_limit = 10
fl = open('lyrics_train_data.txt', 'a')
current_artist_found = False
current_song_found = False
for genre in song_info:
    song_quota_reached = False
    genre_song_count = 0
    songs_genre = ['', genre]
    
    artist_info = song_info[genre]

    for (name, a_id), songs in artist_info.items():
        #continuation after hanging for script
        if not current_artist_found:
            if name.replace(' ','').lower() != 'innercircle':
                continue
            else:
                current_artist_found = True
                print('artist found')
        
        current_fail_count = 0
        for song in songs:
            #continuation after hanging for script
            if not current_song_found:
                if song.replace(' ','').lower() != 'tiredfelickweedinabush':
                    continue
                else:
                    print('Tired song found')
                    current_song_found = True
                    continue
            artist_name = name.split(' (')[0]
            artist_name = re.sub(r'[^A-Za-z0-9 ]+', '', artist_name)
            artist_name = artist_name.replace('  ', '')
            artist_name = artist_name.replace(' ', '')
            artist_name = artist_name.lower()

            song_name = song.split(' (')[0]
            song_name = re.sub(r'[^A-Za-z0-9 ]+', '', song_name)
            song_name = song_name.replace('  ', '')
            song_name = song_name.replace(' ', '')
            song_name = song_name.lower()
        
            azlyrics_url = 'http://www.azlyrics.com/lyrics/' + artist_name + '/' + song_name + '.html'
            print(azlyrics_url + ' ...')
            
            payload = {'api_key': 'b5bcd3b27a08ef14160145e2170fffa1', 'url': azlyrics_url}
            page = requests.get('http://api.scraperapi.com', params=payload)
            soup = BeautifulSoup(page.content, "html.parser")
            
            lyrics = soup.find('div', class_='ringtone')
            if lyrics == None:
                #artist is probably not found in AZ Lyrics
                current_fail_count += 1
                if current_fail_count >= failed_song_limit:
                    break
                    
                print('Not found')
                continue
            else:
                current_fail_count = 0
            lyrics = lyrics.findNext('div')
            lyrics = str(lyrics)
            lyrics = lyrics.split('-->')[1]
            lyrics = lyrics.replace('<br/>', '')
            lyrics = lyrics.replace('</div>', '')
            lyrics = lyrics.replace('\n', ' ')

            if len(lyrics) != 0:
                fl.write(lyrics + '\n')
                fl.flush()
                #songs_genre[0] = songs_genre[0] + lyrics
            
            genre_song_count += 1
            if genre_song_count >= songs_per_genre:
                song_quota_reached = True
                break
        
        if song_quota_reached:
            break
    fl.write(genre + '\n')
    fl.flush()
        
fl.close()

artist found
Tired song found
http://www.azlyrics.com/lyrics/innercircle/sinners.html ...
Not found
http://www.azlyrics.com/lyrics/innercircle/rainfromthesky.html ...
Not found
http://www.azlyrics.com/lyrics/innercircle/smoke.html ...
Not found
http://www.azlyrics.com/lyrics/innercircle/ivegotthehandle.html ...
Not found
http://www.azlyrics.com/lyrics/innercircle/standingfirm7mix.html ...
Not found
http://www.azlyrics.com/lyrics/innercircle/tenementyardinamajor.html ...
Not found
http://www.azlyrics.com/lyrics/innercircle/incoldblood.html ...
Not found
http://www.azlyrics.com/lyrics/innercircle/dislife.html ...
Not found
http://www.azlyrics.com/lyrics/innercircle/whadatfah.html ...
Not found
http://www.azlyrics.com/lyrics/innercircle/drumsong.html ...
http://www.azlyrics.com/lyrics/burningspear/doorpeep.html ...
Not found
http://www.azlyrics.com/lyrics/burningspear/socialliving.html ...
Not found
http://www.azlyrics.com/lyrics/burningspear/creationrebel.html ...
Not found
http://www.az

In [ ]:
#http://www.azlyrics.com/lyrics/ziggymarley/gardensongofmiracles.html ...

In [35]:
fl.flush()